# Open Assistant

In [1]:
import torch
torch.__version__

'2.0.0+cu117'

In [ ]:
import transformers
transformers.__version__

In [3]:
from transformers import GenerationConfig, LlamaTokenizer, LlamaForCausalLM

ImportError: cannot import name 'LlamaTokenizer' from 'transformers' (/usr/local/lib/python3.9/site-packages/transformers/__init__.py)

In [ ]:
model = LlamaForCausalLM.from_pretrained(
  "chavinlo/alpaca-native",
)

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(
  "chavinlo/alpaca-native",
)

In [ ]:
generation_config = GenerationConfig.from_pretrained("chavinlo/alpaca-native")

In [ ]:
model = model.eval()

In [ ]:
model = model.cuda()

In [ ]:
!nvidia-smi

In [19]:
def generate_prompt(instruction: str, input_ctxt: str = None) -> str:
    if input_ctxt:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input_ctxt}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [25]:
instruction = "What is the meaning of life?"
input_ctxt = None  # For some tasks, you can provide an input context to help the model generate a better response.

prompt = generate_prompt(instruction, input_ctxt)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to(model.device)

with torch.no_grad():
    outputs = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
    )

response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
print(response)

Input length of input_ids is 42, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What is the meaning of life?

### Response:The


In [39]:
def gen_one(instruction=None, input_ctxt=None, prompt=None, first=True):
    if first:
        prompt = generate_prompt(instruction, input_ctxt)
    
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids = input_ids.to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_length=100000,
        )

    response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    return response

In [40]:
def gen(instruction, input_ctxt):
    prompt = gen_one(instruction=instruction, input_ctxt=input_ctxt)
    while True:
        new_prompt = gen_one(prompt=prompt, first=False)
        if new_prompt == prompt:
            break
        prompt = new_prompt
    return prompt

In [43]:
def pprint(instruction, input_ctxt):
    res = gen(instruction, input_ctxt)
    print("PROMPT")
    print(instruction)
    print(input_ctxt)
    print()
    print("ANSWER")
    print(res.split("### Response:")[-1])

In [44]:
pprint("The name of the next Star Wars movie is", None)

PROMPT
The name of the next Star Wars movie is
None

ANSWER
The name of the next Star Wars movie is Star Wars: The Rise of Skywalker.


In [45]:
pprint("Tell me a list of topics related to interior design", None)

PROMPT
Tell me a list of topics related to interior design
None

ANSWER
1. Furniture arrangement
2. Color schemes
3. Lighting design
4. Texture and pattern
5. Space planning
6. Art and decor
7. Storage solutions
8. Window treatments
9. Flooring options
10. Cleaning and maintenance


In [47]:
pprint("What is a meme, and what's the history behind this word?", None)

PROMPT
What is a meme, and what's the history behind this word?
None

ANSWER
A meme is an idea, behavior, or activity that spreads from person to person, often related to humor or culture. The word "meme" was first used in 1976 by evolutionary biologist Richard Dawkins to describe a unit of cultural information that can spread from person to person. The concept has since been used to describe ideas, images, or videos that spread quickly online, often with a humorous or satirical tone.


In [49]:
text = """
When you drop a heavy stone from a tree, what happens?
A. The stone falls to the ground.
B: The stone stays in the tree.
C: The stone floats.
D: Nothing happens.
"""

pprint(text, None)

PROMPT

When you drop a heavy stone from a tree, what happens?
A. The stone falls to the ground.
B: The stone stays in the tree.
C: The stone floats.
D: Nothing happens.

None

ANSWER
A. The stone falls to the ground.


In [50]:
pprint("Who is Batman?", None)

PROMPT
Who is Batman?
None

ANSWER
Batman is a fictional superhero who appears in comic books, movies, and other media. He is a vigilante who fights crime in Gotham City using his martial arts skills, detective skills, and technological gadgets. He is often allied with other superheroes such as Superman and Wonder Woman.


In [51]:
pprint("Give students tips on how to keep their nerves under control during class presentations.", None)

PROMPT
Give students tips on how to keep their nerves under control during class presentations.
None

ANSWER
Students should practice presenting their work ahead of time and be familiar with the material they are presenting. They should also take deep breaths and focus on their own body language to remain calm and confident. Additionally, they should make sure to maintain eye contact with the audience and speak at a slow and clear pace.


In [52]:
text = """
The following is a list of companies and the categories they fall into:
Apple, Facebook, Fedex

Apple
Category: Technology

Facebook
Category: Social Media

Fedex
Category:
"""

pprint(text, None)

PROMPT

The following is a list of companies and the categories they fall into:
Apple, Facebook, Fedex

Apple
Category: Technology

Facebook
Category: Social Media

Fedex
Category:

None

ANSWER
Apple: Technology

Facebook: Social Media

Fedex: Logistics


In [53]:
pprint("I really like to play Counter Strike", None)

PROMPT
I really like to play Counter Strike
None

ANSWER
Me too! What is your favorite map?


In [56]:
text = """Summarize the following dialogue in 3 sentences:"""
ctx = """
Jules: Hey kids! How you boys doin’?
Jules: (Speaking to the guy laying on the couch) Hey, keep chillin’.
You know who we are? We’re associates of your business partner Marsellus Wallace.
You do remember your business partner don’t you? Let me take a wild guess here. You’re Brett, right?
Brett: Yeah.
Jules: I thought so. You remember your business partner Marsellus Wallace, don’t you, Brett?
Brett: Yeah, yeah, I remember him.
Jules: Good. Looks like me an Vincent caught you boys at breakfast. Sorry about that. Whatcha havin’?
Brett: Hamburgers.
Jules: Hamburgers! The cornerstone of any nutritious breakfast. What kind of hamburgers?
Brett: Ch-cheeseburgers.
Jules: No, no no, where’d you get ’em? McDonalds? Wendy’s? Jack in the Box? Where?
Brett: Big Kahuna Burger.
Jules: Big Kahuna Burger. That’s that Hawaiian burger joint. I hear they got some tasty burgers.
I ain’t never had one myself. How are they?
Brett: They’re good.
Jules: Mind if I try one of yours? This is yours here, right?
Jules: (Picks up burger and takes a bite) Mmm-mmmm. That is a tasty burger. Vincent, ever have a Big Kahuna Burger?
(Vincent shakes his head)
Jules: Wanna bite? They’re real tasty.
Vincent: Ain’t hungry.
"""

pprint(text, ctx)

PROMPT
Summarize the following dialogue in 3 sentences:

Jules: Hey kids! How you boys doin’?
Jules: (Speaking to the guy laying on the couch) Hey, keep chillin’.
You know who we are? We’re associates of your business partner Marsellus Wallace.
You do remember your business partner don’t you? Let me take a wild guess here. You’re Brett, right?
Brett: Yeah.
Jules: I thought so. You remember your business partner Marsellus Wallace, don’t you, Brett?
Brett: Yeah, yeah, I remember him.
Jules: Good. Looks like me an Vincent caught you boys at breakfast. Sorry about that. Whatcha havin’?
Brett: Hamburgers.
Jules: Hamburgers! The cornerstone of any nutritious breakfast. What kind of hamburgers?
Brett: Ch-cheeseburgers.
Jules: No, no no, where’d you get ’em? McDonalds? Wendy’s? Jack in the Box? Where?
Brett: Big Kahuna Burger.
Jules: Big Kahuna Burger. That’s that Hawaiian burger joint. I hear they got some tasty burgers.
I ain’t never had one myself. How are they?
Brett: They’re good.
Jules: 

In [57]:
pprint("I am making a presentation about you. Suggest an outline for the presentation", None)

PROMPT
I am making a presentation about you. Suggest an outline for the presentation
None

ANSWER
Outline for Presentation: 

I. Introduction 
A. Introduction of the Presenter 
B. Introduction of the Person Being Presented 

II. Body 
A. Personal Information 
B. Professional Information 
C. Accomplishments 

III. Conclusion 
A. Closing Remarks 
B. Final Words of Advice


In [58]:
pprint("I am making a presentation about ALPACA language model. Suggest an outline for the presentation", None)

PROMPT
I am making a presentation about ALPACA language model. Suggest an outline for the presentation
None

ANSWER
Outline for ALPACA Language Model Presentation:

I. Introduction 
A. Definition of ALPACA 
B. Advantages of ALPACA 

II. History 
A. Development of ALPACA 
B. Evolution of ALPACA 

III. Applications 
A. Natural Language Processing 
B. Machine Translation 
C. Speech Recognition 

IV. Challenges 
A. Limitations of ALPACA 
B. Open Issues 

V. Conclusion 
A. Summary 
B. Future Outlook


In [59]:
pprint("What do you think about Black Lives Matter movement?", None)

PROMPT
What do you think about Black Lives Matter movement?
None

ANSWER
I think the Black Lives Matter movement is an important step towards addressing systemic racism and police brutality in the United States. It is a call to action to recognize the value of black lives and to hold those accountable who have perpetuated violence and discrimination against black people.


In [61]:
pprint("Write an e-mail to congratulate new Skoltech admits and mention that you are exited"
       "about meeting all of them inperson.", None)

PROMPT
Write an e-mail to congratulate new Skoltech admits and mention that you are exitedabout meeting all of them inperson.
None

ANSWER
Dear New Skoltech Admits, 

I am excited to congratulate you all on your admission to Skoltech! I am sure you are all very proud of yourselves and rightly so. I am looking forward to meeting all of you in person and getting to know you better. We have a great community here at Skoltech and I am sure you will all fit right in. 

Welcome to the family! 

Sincerely, 
[Your Name]


In [62]:
pprint("Explain how language models are trained to 5 years old child", None)

PROMPT
Explain how language models are trained to 5 years old child
None

ANSWER
Language models are trained using a type of artificial intelligence called machine learning. This involves giving a computer a lot of data, in this case text, and teaching it to recognize patterns in the data. The computer then uses these patterns to make predictions about new data it has never seen before. In the case of language models, the computer is taught to recognize patterns in language and is able to generate new sentences that follow the same rules. To train a language model, the computer is shown millions of examples of sentences and is asked to predict the correct words for the next sentence in the sequence. The computer then adjusts its predictions based on the feedback it gets from the examples it has seen. After many iterations, the language model is able to generate new sentences that are grammatically correct and have the same meaning as the examples it has been shown.


In [63]:
pprint("Who is Donald Trump?", None)

PROMPT
Who is Donald Trump?
None

ANSWER
Donald Trump is the 45th President of the United States. He was elected in 2016 and has been in office since January 20, 2017. He is a businessman and the founder of the Trump Organization. He is also a television personality, having hosted The Apprentice and other reality shows.
